# What
Our code is a mess. This is just to not have to run everything all the time. Allows us to focus on the data we have instead of trying to get even more messy data.

This should be used as a default and copies should be done to build on top. The idea of this is that it's a basic AF readers so that actual analysis can be done.

In [113]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import statistics
import math

# networkx
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

# For semantic similarity
from urllib.parse import unquote
from transformers import BertTokenizer
from transformers import BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Python functions in .py file to read data
import data_readers

import warnings
from tqdm import TqdmWarning
warnings.filterwarnings('ignore', category=TqdmWarning)

In [114]:
# The links and edges
wikispeedia = data_readers.read_wikispeedia_graph()

# The finished paths
finished_paths = data_readers.read_finished_paths()

# The unfinished paths
unfinished_paths = data_readers.read_unfinished_paths()

# DF of all articles and their categories
categories = data_readers.read_categories()

# Searching for the string of a given article. It has to be formatted like the article name
# Which shouldn't be a problem, as we'll probably usually retrieve them internally
obi_wan_text = data_readers.plaintext_article_finder('Obi-Wan_Kenobi')

source_target_info = data_readers.source_target_paths_information()

In [115]:
# This is a last step, of making these three datasets have consistent column names and order
# as well as guaranteeing that the index order is conistent with both elements
# Even though the boolean column is weird, it's a boolean so that's all good

# FUCK IT. We don't touch the decoded version, that's too painful!

col_names = ['source', 'target', 'reached', 'length', 'visited']

import ast

carlos_df = pd.read_csv('machine_outputs/data_carlos.csv')
carlos_df.columns = col_names

carlos_df['visited'] = carlos_df['visited'].str.strip('\\').str.strip('{}')
carlos_df['visited'] = carlos_df['visited'].apply(lambda x: ast.literal_eval(x))

carol_df = pd.read_csv('machine_outputs/carolOutput.csv')
carol_df.columns = col_names

carol_df['visited'] = carol_df['visited'].str.strip('\\').str.strip('{}')
carol_df['visited'] = carol_df['visited'].apply(lambda x: ast.literal_eval(x))

landmark_df = pd.read_csv('machine_outputs/landmark_method_results.csv')

# Landmark_df is the only one with substantive change
landmark_df.drop('Unnamed: 0', axis=1, inplace=True)
landmark_df['reached'] = True
landmark_df.columns = ['source', 'target', 'length', 'visited', 'reached']
landmark_df = landmark_df[col_names]
# Due to the way length was counted here, I need to update to increase by one
landmark_df['length'] = landmark_df['length'] + 1

landmark_df['visited'] = landmark_df['visited'].str.strip('\\').str.strip('{}')
landmark_df['visited'] = landmark_df['visited'].apply(lambda x: ast.literal_eval(x))

Modify given datasets

In [116]:
unfinished_paths['source'] = unfinished_paths['path'].str.split(';').apply(lambda x: x[0])
unfinished_paths = unfinished_paths[['source', 'target', 'path', 'type']]


In [117]:
def add_category_of_target_article(df_to_add_category_to) -> pd.DataFrame:
    categories[categories['article'].duplicated()]
    categories['article'] = categories['article'].drop_duplicates()
    sub_categories = categories['categories'].str[8:].str.split('.')
    category_depth_1 = sub_categories.apply(lambda x: x[0])
    categories['category_target'] = category_depth_1

    new_df = pd.merge(df_to_add_category_to, categories, left_on = 'target', right_on= 'article', how = 'left')
    new_df = new_df.drop(columns = ['article', 'categories'])
    return new_df

landmark_df = add_category_of_target_article(landmark_df)

Can machine do some of the unfinished paths that man couldn’t?


In [121]:
def find_unfinished_paths_from_man_and_machine(human_df, machine_df) -> pd.DataFrame:
    return pd.merge(unfinished_paths, machine_df, how = 'inner', on = ['target', 'source'])

find_unfinished_paths_from_man_and_machine(unfinished_paths, landmark_df)['target'].value_counts()

target
Telephone              1052
Bean                    944
Zebra                   884
Viking                  728
Wood                     82
                       ... 
Pythagorean_theorem       1
Graphic_novel             1
Brass_instrument          1
Svalbard                  1
Owl                       1
Name: count, Length: 863, dtype: int64